### help flask omG

define all of the functions i have made

In [4]:
from PIL import Image
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import datetime
from werkzeug.utils import secure_filename
import random
import time
import folium

session = requests.session()
known_proxy_ip = '109.173.86.43:8000'
proxy = {'http': known_proxy_ip, 'https': known_proxy_ip}

with open('errors.txt', 'w') as f:
    pass

def get_page(url): #скачивает страницу в soup
    
    user_agent = UserAgent().chrome
    try:
        response = session.get(url, headers={'User-Agent':user_agent})
    except Exception as err:
        with open('errors.txt', 'a') as f:
            f.write(f'{err}\t{url}\n')
    #response = session.get(url, proxies=proxy)
    soup = BeautifulSoup(response.text, 'html.parser')
    time.sleep(random.random())
    
    return soup

def get_wiki_image(name, link='https://en.wikipedia.org/wiki/'):
    wiki = get_page(link+name)
    l = wiki.find('img')['src']
    user_agent = UserAgent().chrome
    response = requests.get('https:'+l, headers={'User-Agent':user_agent}, stream=True)
    return Image.open(response.raw)

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from flask import Flask
from flask import render_template
import sys
import os

from flask import request, redirect, url_for
from sqlalchemy import func
from flask_sqlalchemy import SQLAlchemy
from password import password

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///motus.db'
db = SQLAlchemy(app)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True
keyword = password

results = {
    'North America':{
        'mammals':125530,
        'bats':100460,
        'birds':16300,
        'insects':252456
    },
    'Europe':{
        'mammals':129470,
        'bats':123690,
        'birds':5970,
        'insects':1000069
    },
    'South':{
        'mammals':110930,
        'bats':121310,
        'birds':7060,
        'insects':263363
    },
}

result_print = {
    125530: {'result':'Indiana bat',
            'text':'A little bat native to the US! Are you an American citizen?'},
    100460:{'result':'Pipistrelle',
            'text':'A cute tiny bat! Also, your name is Italian but you populate vast areas of Europe and North America.'},
    16300:{'result':'Sprague\'s Pipit',
            'text':'What a cute name! Carries the name of a botanical artist Isaac Sprauge. Beautiful singing voice as well!'},
    252456:{'result':'Monarch butterfly',
            'text':'A gorgeous butterfly, what else to say?'},
    129470:{'result':'Human',
            'text':'Yes, Motus tracks humans as well. They\'re watching you (in Europe and America).'},
    123690:{'result':'Serotine bat',
            'text':'You misread it as \'serotonin bat\', didn\'t you? That is because you bring happiness everywhere you go!'},
    5970:{'result':'Rock pigeon',
            'text':'The legendary city bird. Have you considered feeding your local pigeons? Hurry to befriend your local pigeon gang.'},
    1000069:{'result':'Buff-tailed bumblebee',
            'text':'Flying around, helping the flowers bloom. БЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖ!'},
    110930:{'result':'Northern quoll',
            'text':'What an adorable carnivorous cat! An Australian native too.'},
    121310:{'result':'Orange leaf-nosed bat',
            'text':'Look at your bright orange fur! A proud citizen of Australia, send my regards to your kangaroo friends.'},
    7060:{'result':'Yellow-billed cuckoo',
            'text':'Americans call you rain crow or storm crow. But you\'re not a crow... Coo-ckoo, coo-ckoo!'},
    263363:{'result':'Hymenopteran',
            'text':'A bee, a wasp or something else? Stinging others or pollinating flowers? You tell me!'},
}

conn = sqlite3.connect('motus.db')
cur = conn.cursor()

#создать таблицы в базе данных

cur.execute("""
CREATE TABLE IF NOT EXISTS responses 
(id INTEGER PRIMARY KEY AUTOINCREMENT, q1 text, q2 text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS posts 
(id INTEGER PRIMARY KEY AUTOINCREMENT, title text, text text, image text, datetime text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS searches 
(id INTEGER PRIMARY KEY AUTOINCREMENT, user_id integer, search text) 
""")

conn.commit()
conn.close()

path = ''

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/credits')
def credits():
    return render_template('credits.html')

@app.route('/test')
def test():
    return render_template('test.html')

@app.route('/test_q1')
def test_q1():
    return render_template('test_q1.html')

@app.route('/test_q2')
def test_q2():
    return render_template('test_q2.html')

class Response(db.Model):
    __tablename__ = 'responses'
    id = db.Column(db.Integer, primary_key=True)
    q1 = db.Column(db.Text)
    q2 = db.Column(db.Text)
    
class Post(db.Model):
    __tablename__ = 'posts'
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.Text)
    text = db.Column(db.Text)
    image = db.Column(db.Text)
    datetime = db.Column(db.Text)
    
class Search(db.Model):
    __tablename__ = 'searches'
    id = db.Column(db.Integer, primary_key=True)
    continent = db.Column(db.Text)
    group = db.Column(db.Text)
    species = db.Column(db.Text)
    
@app.route('/process', methods=['GET', 'POST'])
def answer_process():
    if request.method == 'POST' or not request.args:
        return redirect(url_for('test'))
    q1 = request.args.get('q1')
    q2 = request.args.get('q2')
    response = Response(
        q1=q1,
        q2=q2
    )
    db.session.add(response)
    db.session.commit()
    db.session.refresh(response)
    if q2:
        nex = 'test_result'
    else: 
        nex = 'test_q2'
    return redirect(url_for(nex))

@app.route('/test_result')
def test_result():
    #q1 = db.session.query(Response.q1).where(Response.q1 != None).first()[0]
    #q2 = db.session.query(Response.q2).where(Response.q2 != None).first()[0]
    q1 = db.session.query(Response.q1).order_by(Response.id.desc()).where(Response.q1 != None).first()[0]
    q2 = db.session.query(Response.q2).order_by(Response.id.desc()).where(Response.q2 != None).first()[0]
    i = results[q1][q2]
    title = result_print[i]['result']
    text = result_print[i]['text']
    image = title + '.png'
    if image not in os.listdir('static/'):
        get_wiki_image(title).save('static/' + image)
    link = 'https://en.wikipedia.org/wiki/' + title
    return render_template('test_result.html', title=title, text=text, image=image, link=link)

@app.route('/map_form_q1')
def map_form():
    return render_template('map_form_q1.html')

@app.route('/map_render')
def maps():
    return render_template('map_render.html')

@app.route('/map')
def maps1():
    return render_template('recv_map.html', lat=[20,21])

@app.route('/blog')
def blog():
    return render_template('blog.html')

@app.route('/continent', methods=['GET', 'POST'])
def continent():
    if request.method == 'POST':
        continent = request.form.get('continent')
        query = f'''SELECT DISTINCT group__
                FROM tag_deployments
                    JOIN motus_projects ON tag_deployments.tagProjectID = motus_projects.projectID
                    JOIN species ON species.id = tag_deployments.speciesID
                    JOIN receiver_deployments ON motus_projects.projectID = receiver_deployments.recvProjectID
                WHERE continent='{continent}'
                '''
        
        groups = [ele[0] for ele in db.engine.execute(query)]
        
    return render_template('map_form_q2.html', groups=groups, continent=continent)

@app.route('/group', methods=['GET', 'POST'])
def group():
    if request.method == 'POST':
        continent = request.form.get('continent')
        group = request.form.get('group')
        query = f'''SELECT DISTINCT english
                FROM tag_deployments
                    JOIN motus_projects ON tag_deployments.tagProjectID = motus_projects.projectID
                    JOIN species ON species.id = tag_deployments.speciesID
                    JOIN receiver_deployments ON motus_projects.projectID = receiver_deployments.recvProjectID
                WHERE continent='{continent}' AND group__='{group}'
                '''
        
        species = [ele[0] for ele in db.engine.execute(query)]
    return render_template('map_form_q3.html', group=group, species=species, continent=continent)

@app.route('/species', methods=['GET', 'POST'])
def species():
    if request.method == 'POST':
        continent = request.form.get('continent')
        group = request.form.get('group')
        species = request.form.get('species')
        query = f'''SELECT DISTINCT receiver_deployments.recvDeployID
                FROM tag_deployments
                    JOIN motus_projects ON tag_deployments.tagProjectID = motus_projects.projectID
                    JOIN species ON species.id = tag_deployments.speciesID
                    JOIN receiver_deployments ON motus_projects.projectID = receiver_deployments.recvProjectID
                WHERE continent='{continent}' AND group__='{group}' AND english='{species}'
                '''
        sp_id = [ele for ele in db.engine.execute(query)]
        
        query = f'''SELECT DISTINCT gpsLat, gpsLon
                FROM receivers
                WHERE recvDeployID={sp_id[0][0]}'''
        
        locations = [ele for ele in db.engine.execute(query)]
        if len(locations) == 0:
            return render_template('map_res_error.html')
        color = 'green'

        m = folium.Map(location=locations[0])
        folium.vector_layers.PolyLine(locations, color=color).add_to(m)

        m.save('templates/route_map.html')
    return render_template('map_res.html')

@app.route('/route_map')
def route_map():
    return render_template('route_map.html')

@app.route('/post', methods=['GET', 'POST'])
def post():
    if request.method == 'POST':
        password = request.form.get('password')
        if password != keyword:
            return render_template('error.html')
        image_file = request.files['image']
        if image_file.filename != '':
            image = image_file.filename
            fn = secure_filename(image_file.filename)
            image_file.save(os.path.join('static/', fn))
        title = request.form.get('title')
        text = request.form.get('text')
        #image = request.args.get('image')
    date = str(datetime.datetime.now())[:-7]
    post = Post(
            title=title,
            text=text,
            image=fn,
            datetime=date
        )
    db.session.add(post)
    db.session.commit()
    db.session.refresh(post)
    return render_template('thanks.html')

@app.route('/show_post')
def show_post():
    '''title = db.session.query(Post.title).first()[0]
    text = db.session.query(Post.text).first()[0]
    image = db.session.query(Post.image).first()[0]
    datetime = db.session.query(Post.datetime).first()[0]'''
    return redirect(url_for('news.html')) 

@app.route('/news')
def news():
    posts = db.session.query(Post.title, Post.text, Post.image, Post.datetime).all()
    return render_template('news.html', posts=reversed(posts))

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/shikunova/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Mar/2022 22:39:21] "GET /news HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 22:39:24] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 22:39:25] "GET /test_q1 HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 22:39:29] "GET /process?q1=North+America HTTP/1.1" 302 -
127.0.0.1 - - [26/Mar/2022 22:39:29] "GET /test_q2 HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 22:39:30] "GET /process?q2=insects HTTP/1.1" 302 -
127.0.0.1 - - [26/Mar/2022 22:39:33] "GET /test_result HTTP/1.1" 200 -
127.0.0.1 - - [26/Mar/2022 22:39:33] "GET /static/Monarch%20butterfly.png HTTP/1.1" 200 -


In [62]:
con = sqlite3.connect('motus.db')  # подключение
cur = con.cursor()  # курсор

In [42]:
query = '''INSERT INTO responses (q2)
VALUES (1);
'''

cur.execute(query)

In [63]:
query = '''DROP TABLE IF EXISTS posts;
'''

cur.execute(query)
cur.fetchall()

[]

In [99]:
con.close()

In [60]:
str(datetime.datetime.now())[:-7]

'2022-03-25 15:39:42'